## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [3]:
# Load the dataset
import pandas as pd
data=pd.read_csv('D:\\UDEMY_course\\kindle_reviews.csv\\kindle_reviews.csv')
data.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [4]:
df=data[['reviewText','overall']]
df=df.rename(columns={'overall':'rating'})
df.head()

,reviewText,rating
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [5]:
df.shape

(982619, 2)

In [6]:
# Assuming your DataFrame is named df
df_new = df.sample(frac=0.002, random_state=42)  # Adjust the frac as needed

# Check the new shape
print(df_new.shape)

(1965, 2)


In [7]:
## Missing Values
df_new.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [16]:
df_new.dropna(inplace=True)

In [17]:
df_new.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [8]:
df_new['rating'].unique()

array([5, 3, 4, 2, 1], dtype=int64)

In [9]:
df_new['rating'].value_counts()

rating
5    1174
4     481
3     193
2      77
1      40
Name: count, dtype: int64

In [85]:
## Preprocessing And Cleaning

In [10]:
## postive review is 1 and negative review is 0
df_new['rating']=df_new['rating'].apply(lambda x:0 if x<=3 else 1)

In [11]:
df_new['rating'].value_counts()

rating
1    1655
0     310
Name: count, dtype: int64

In [12]:
## 1. Lower All the cases
df_new['reviewText']=df_new['reviewText'].str.lower()

In [13]:
df_new.head()

,reviewText,rating
869697,arc provided by author in exchange for an hone...,1
760913,wild ride by nancy warrenchanging gears series...,1
159841,"well thought out story, with many things going...",1
868915,this is book four of a five part serial. by n...,0
980703,i really enjoyed this book. it kept me interes...,1


In [14]:
import re
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

In [15]:
from bs4 import BeautifulSoup

In [41]:
!pip install lxml

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   -- ------------------------------------- 0.2/3.8 MB 11.5 MB/s eta 0:00:01
   ----- ---------------------------------- 0.5/3.8 MB 8.1 MB/s eta 0:00:01
   --------- ------------------------------ 0.9/3.8 MB 7.0 MB/s eta 0:00:01
   ---------- ----------------------------- 1.0/3.8 MB 7.0 MB/s eta 0:00:01
   --------------- ------------------------ 1.5/3.8 MB 6.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.9/3.8 MB 6.3 MB/s eta 0:00:01
   ---------------------- ----------------- 2.2/3.8 MB 6.6 MB/s eta 0:00:01
   -------------------------- ------------- 2.5/3.8 MB 6.6 MB/s eta 0:00:01
   ----------------------------- ---------- 2.8/3.8 MB 6.7 MB/s eta 0:00:01
   ------------------------------- -------- 3.0/3.8 MB 6.5 MB/s eta 0:00:01
   -------------------------------- ------- 3.1/3.8 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 3.3/3.8 MB 5.9 MB/s eta 0:00:01
   ---------------


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
## Removing special characters
df_new['reviewText']=df_new['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df_new['reviewText']=df_new['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df_new['reviewText']=df_new['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df_new['reviewText']=df_new['reviewText'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
## Remove any additional spaces
df_new['reviewText']=df_new['reviewText'].apply(lambda x: " ".join(x.split()))


In [74]:
df_new.head()

,reviewText,rating
869697,arc provided author exchange honest reviewthis...,1
760913,wild ride nancy warrenchanging gears seriesdun...,1
159841,well thought story many things going time alie...,1
868915,book four five part serial suspense highest im...,0
980703,really enjoyed book kept interested page one w...,1


In [17]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [18]:
lemmatizer=WordNetLemmatizer()

In [19]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [20]:
df_new['reviewText']=df_new['reviewText'].apply(lambda x:lemmatize_words(x))

In [21]:
df_new.head()

,reviewText,rating
869697,arc provided author exchange honest reviewthis...,1
760913,wild ride nancy warrenchanging gear seriesdunc...,1
159841,well thought story many thing going time alien...,1
868915,book four five part serial suspense highest im...,0
980703,really enjoyed book kept interested page one w...,1


In [22]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_new['reviewText'],df_new['rating'],
                                              test_size=0.20)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [89]:
#from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf=TfidfVectorizer(max_features=2500)
#X_train_tfidf=tfidf.fit_transform(X_train).toarray()
#X_test_tfidf=tfidf.transform(X_test).toarray()

In [24]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
#nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [27]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [110]:
#y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [28]:
confusion_matrix(y_test,y_pred_bow)

array([[ 15,  36],
       [ 69, 273]], dtype=int64)

In [29]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.732824427480916


In [115]:
#confusion_matrix(y_test,y_pred_tfidf)

array([[502, 317],
       [687, 894]], dtype=int64)

In [114]:
#print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5816666666666667


In [30]:
### WORD2VEC applying on this project Assignment
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [31]:
corpus=df_new['reviewText']

In [32]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [33]:
words

[['arc',
  'provided',
  'author',
  'exchange',
  'honest',
  'reviewthis',
  'first',
  'time',
  'read',
  'book',
  'miranda',
  'charles',
  'lastthis',
  'book',
  'broken',
  'heart',
  'twisted',
  'story',
  'lie',
  'scared',
  'confused',
  'lover',
  'zach',
  'rebecca',
  'met',
  'hired',
  'private',
  'nurse',
  'grandmother',
  'surgery',
  'dated',
  'four',
  'month',
  'rebecca',
  'said',
  'three',
  'little',
  'word',
  'zach',
  'could',
  'say',
  'back',
  'rebecca',
  'break',
  'chance',
  'tomonths',
  'later',
  'hired',
  'join',
  'grandmother',
  'month',
  'long',
  'cruise',
  'birthday',
  'nurse',
  'grandmother',
  'also',
  'friend',
  'grandson',
  'joining',
  'birthday',
  'celebration',
  'cruise',
  'around',
  'rebecca',
  'zach',
  'work',
  'fear',
  'find',
  'love',
  'againa',
  'sexy',
  'quick',
  'read',
  'able',
  'put'],
 ['wild',
  'ride',
  'nancy',
  'warrenchanging',
  'gear',
  'seriesduncan',
  'forbes',
  'professor',
  'h

In [34]:
import gensim
## Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words)

In [35]:
words[0]

['arc',
 'provided',
 'author',
 'exchange',
 'honest',
 'reviewthis',
 'first',
 'time',
 'read',
 'book',
 'miranda',
 'charles',
 'lastthis',
 'book',
 'broken',
 'heart',
 'twisted',
 'story',
 'lie',
 'scared',
 'confused',
 'lover',
 'zach',
 'rebecca',
 'met',
 'hired',
 'private',
 'nurse',
 'grandmother',
 'surgery',
 'dated',
 'four',
 'month',
 'rebecca',
 'said',
 'three',
 'little',
 'word',
 'zach',
 'could',
 'say',
 'back',
 'rebecca',
 'break',
 'chance',
 'tomonths',
 'later',
 'hired',
 'join',
 'grandmother',
 'month',
 'long',
 'cruise',
 'birthday',
 'nurse',
 'grandmother',
 'also',
 'friend',
 'grandson',
 'joining',
 'birthday',
 'celebration',
 'cruise',
 'around',
 'rebecca',
 'zach',
 'work',
 'fear',
 'find',
 'love',
 'againa',
 'sexy',
 'quick',
 'read',
 'able',
 'put']

In [64]:
import gensim.downloader as api

#wv = api.load('word2vec-google-news-300')

In [37]:
import numpy as np
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [38]:
from tqdm import tqdm
#apply for the entire sentences
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 1965/1965 [00:04<00:00, 395.32it/s]


In [39]:
len(X)

1965

In [41]:
X[1]

array([-0.15737657,  0.23528935,  0.13519898,  0.06284352,  0.24778923,
       -0.58964527,  0.11020012,  0.6131555 , -0.30307627, -0.15178514,
       -0.15790899, -0.4042848 , -0.14804377,  0.12438317,  0.1238591 ,
       -0.24681763,  0.11081386, -0.2772401 , -0.03997329, -0.6656985 ,
        0.13104844,  0.24623092,  0.20889136, -0.10364357, -0.05904686,
       -0.03495906, -0.23728862, -0.2508821 , -0.439096  ,  0.03132483,
        0.21294396, -0.01382344,  0.2004543 , -0.31527424, -0.02571858,
        0.3968328 , -0.08479067, -0.32581404, -0.19218823, -0.60708904,
       -0.00378171, -0.39923465, -0.13214524,  0.10025679,  0.35656428,
       -0.12162136, -0.17953748, -0.15231282,  0.23805533,  0.15034525,
        0.08004931, -0.21756776, -0.04140564, -0.12267987, -0.26413548,
        0.20562328,  0.05927782, -0.12692373, -0.3557939 ,  0.13380304,
        0.07790598,  0.13940063,  0.02945886, -0.09269894, -0.48236915,
        0.2861735 ,  0.09954294,  0.2748229 , -0.40048644,  0.36

In [42]:
y=df_new['rating'].values
y.shape

(1965,)

In [43]:
dfp=X[0].reshape(1,-1).shape
dfp

(1, 100)

In [46]:
## this is the final independent features
#df=pd.DataFrame()
#for i in range(0,len(X)):
#    df=df.append(pd.DataFrame(X[i].reshape(1,-1)),ignore_index=True)

In [45]:
df1=[]
for i in range(0,len(X)):
    df1.append(X[i].reshape(1,-1))#,ignore_index=True
df=pd.DataFrame(np.concatenate(df1, axis=0))

In [47]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.142015,0.209565,0.116797,0.054471,0.213336,-0.530173,0.096664,0.541668,-0.273763,-0.134869,...,0.367469,0.207131,0.170079,0.210984,0.470930,0.274722,0.096924,-0.194457,0.001970,-0.076189
1,-0.157377,0.235289,0.135199,0.062844,0.247789,-0.589645,0.110200,0.613155,-0.303076,-0.151785,...,0.422627,0.225992,0.184970,0.244076,0.522893,0.310399,0.111907,-0.223542,0.002705,-0.079432
2,-0.169701,0.250354,0.140096,0.067244,0.257191,-0.635728,0.117771,0.650816,-0.330434,-0.160301,...,0.444349,0.247335,0.194731,0.254824,0.561555,0.334915,0.117533,-0.228383,0.002520,-0.082123
3,-0.149301,0.223205,0.125391,0.061679,0.223663,-0.567623,0.105939,0.573169,-0.295072,-0.142384,...,0.390910,0.219369,0.177573,0.226351,0.502488,0.300422,0.099817,-0.204336,0.003171,-0.083248
4,-0.176897,0.258459,0.145646,0.074862,0.265498,-0.661192,0.123453,0.677976,-0.345859,-0.169961,...,0.462207,0.261428,0.205412,0.267934,0.588620,0.348095,0.119187,-0.239256,0.000897,-0.091460


In [48]:
df['Output']=y

In [49]:
## Independent Feature
X=df

In [50]:
y=df['Output']

In [51]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [52]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Output
282,-0.156428,0.234536,0.131445,0.062046,0.236024,-0.589273,0.108491,0.604043,-0.302145,-0.150117,...,0.228854,0.185851,0.234126,0.523028,0.310298,0.104940,-0.211793,0.001440,-0.086364,0
1698,-0.143634,0.212951,0.120637,0.057057,0.220411,-0.539071,0.100925,0.556099,-0.278737,-0.137972,...,0.210095,0.167898,0.217958,0.477618,0.283406,0.098396,-0.199625,0.001175,-0.071644,0
788,-0.176451,0.260957,0.148050,0.070924,0.276240,-0.660980,0.126663,0.689807,-0.343806,-0.173662,...,0.254254,0.201672,0.272806,0.586546,0.348181,0.126254,-0.253949,0.003802,-0.082068,0
400,-0.158733,0.233091,0.133131,0.062484,0.242961,-0.587561,0.107434,0.606013,-0.303833,-0.147596,...,0.226378,0.190598,0.240252,0.524102,0.306175,0.111641,-0.218208,-0.000294,-0.083841,1
1859,-0.155496,0.231686,0.131604,0.061669,0.238809,-0.586169,0.108197,0.603139,-0.302170,-0.151235,...,0.225553,0.183907,0.237495,0.520579,0.307586,0.108730,-0.218169,0.002335,-0.081435,1


In [53]:
y_train

282     0
1698    0
788     0
400     1
1859    1
       ..
1939    1
1118    1
1901    1
1797    0
1832    1
Name: Output, Length: 1572, dtype: int64

In [57]:
X_train.columns = X_train.columns.astype(str)

In [58]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [60]:
X_test.columns = X_test.columns.astype(str)

In [61]:
y_pred=classifier.predict(X_test)

In [62]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

1.0


In [63]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        59
           1       1.00      1.00      1.00       334

    accuracy                           1.00       393
   macro avg       1.00      1.00      1.00       393
weighted avg       1.00      1.00      1.00       393

